In [41]:
import os 
from pathlib import Path
import sys
import subprocess
import time
from typing import Optional
from collections import defaultdict 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from IPython.display import display
import latextable
from texttable import Texttable
from arguments_dict import arguments_dict

executables = ['genmc-old', 'genmc-wkmo', 'genmc', 'genmc-xmm']
executable_paths_dict = dict([(exe, Path("executables") / exe) for exe in executables])
tests = dict([(t, Path('litmus') / t / 'variants' / (os.listdir(Path('litmus') / t / 'variants')[0])) for t in os.listdir('litmus') if os.path.isdir(Path('litmus') / t)])
subprocess_timeout = 60 #s

class RunExecutableResult:
    class Timeout:
        pass
    class Ok:
        def __init__(self, output) -> None:
            super().__init__()
            self.output = output
    class Error:
        def __init__(self, retcode, output) -> None:
            super().__init__()
            self.retcode = retcode
            self.output = output

def run_executable(exe_path: Path, args: list[str], test_path: Path) -> RunExecutableResult:
    try:
        result = subprocess.run([exe_path, *args, test_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=subprocess_timeout)
        result_stdout = result.stdout.decode()
        result_stderr = result.stderr.decode()
        if result.returncode == 0:
            return RunExecutableResult.Ok(result_stdout + result_stderr)
        else:
            return RunExecutableResult.Error(result.returncode, result_stdout + result_stderr)
    except subprocess.TimeoutExpired:
        return RunExecutableResult.Timeout()

In [42]:
def get_execution_time(string: str) -> Optional[float]:
    try:
        pattern = "Total wall-clock time: "
        time = string.split(pattern)[1].split('s\n')[0]
        return float(time)
    except:
        return None
assert(get_execution_time(
"""
No errors were detected.
Number of complete executions explored: 3
Total wall-clock time: 0.05s
""") == 0.05)

def get_number_executions(string: str) -> Optional[int]:
    pattern = "Number of complete executions explored: "
    i = string.index(pattern) + len(pattern)
    number_str = ""
    while i < len(string) and string[i].isdigit():
        number_str += string[i]
        i += 1
    try:
        return int(number_str)
    except:
        return None

assert(get_number_executions("abc 123 Number of complete executions explored: 123456") == 123456)

def get_duplicate_executions(string: str) -> Optional[int]:
    try:
        pattern = "Number of duplicate executions explored: "
        i = string.index(pattern) + len(pattern)
        number_str = ""
        while i < len(string) and string[i].isdigit():
            number_str += string[i]
            i += 1
        return int(number_str)
    except:
        return None

assert(get_duplicate_executions("abc 123 Number of duplicate executions explored: 123456") == 123456)

def get_number_lb_races(string: str) -> Optional[int]:
    pattern = "Number of load buffering races explored: "
    try:
        i = string.index(pattern) + len(pattern)
        number_str = ""
        while i < len(string) and string[i].isdigit():
            number_str += string[i]
            i += 1
        return int(number_str)
    except:
        return None

assert(get_number_lb_races("abc 123 Number of load buffering races explored: 123456") == 123456)

class ResultItem:
    def __init__(self, execs, exe_time, dups, lb_races) -> None:
        self.execs = execs
        self.exe_time = exe_time
        self.dups = dups
        self.lb_races = lb_races

def run_and_get_results(exe_path: Path, args: list[str], test_path: Path) -> RunExecutableResult:
    res = run_executable(exe_path, args, test_path)
    if isinstance(res, RunExecutableResult.Ok):
        execs = get_number_executions(res.output)
        exe_time = get_execution_time(res.output)
        dups = get_duplicate_executions(res.output)
        lb_races = get_number_lb_races(res.output)
        if execs is None:
            # print(f"could not get number of executions in {res.output}")
            execs = 0
        if dups is None:
            dups = 0
        if exe_time is None:
            # print(f"could not get duplicate executions in {res.output}")
            exe_time = 0
        return RunExecutableResult.Ok(ResultItem(execs, exe_time, dups, lb_races))
    else:
        return res

In [43]:
timeout = '{\\fontspec{Symbola}\\symbol{"231B}}'
rows = [["Test Name", "Old GenMC Execs",  "Old GenMC Duplicates", "Old GenMC Time", "WMC Execs", "WMC Duplicates", "WMC Time", "GenMC Execs", "GenMC Duplicates", "GenMC Time", "XMM Execs", "XMM Duplicates", "XMM Time", "LB races"]]
for test_name in tests.keys():
    row = [f"{test_name}"]
    for exe in executables:
        res = run_and_get_results(executable_paths_dict[exe], arguments_dict[exe], tests[test_name])
        if isinstance(res, RunExecutableResult.Ok):
            execs = res.output.execs
            exe_time = res.output.exe_time      
            dups = res.output.dups          
            row += [execs, dups, f"{exe_time:.2f}s"]
            if res.output.lb_races is not None:
                row += [res.output.lb_races]
        elif isinstance(res, RunExecutableResult.Timeout):
            row += [timeout] * 3
        elif isinstance(res, RunExecutableResult.Error):
            print(f"{exe} {test_name} error {res.retcode}: {res.output[:100]}")
            # row += [res.output[:60] if res.output != "" else f"error code {res.retcode}"] * 2
            row = None
            break
        else:
            assert(0)
    
    rows.append(row) if row is not None else None

print("rows has len", len(rows))

rows has len 170


In [44]:
data = rows[1:]

for i in range(len(data)):
    data[i][0] = data[i][0].replace("_", "-")

data = [d for d in data if d[1] != d[4] or d[7] != d[10]]

assert('Execs' in rows[0][1])
assert('Execs' in rows[0][4])
assert('Execs' in rows[0][7])
assert('Execs' in rows[0][10])

rows = [rows[0]] + data

In [45]:
exec_rows = [[row[0]] + row[1:-1][0::3] for row in rows]

print([r for r in exec_rows if r[0] == "viktor-relseq"][0])
assert([r for r in exec_rows if r[0] == "viktor-relseq"][0][4] == timeout)
exec_rows = [r for r in exec_rows if r[0] != "viktor-relseq"]

cyclic_rows = [[rows[0][0], "WMC cyclic execs", "XMM cyclic execs"]] + [[row[0], int(row[2]) - int(row[1]), int(row[4]) - int(row[3])] for row in exec_rows[1:]]

data = cyclic_rows[1:]
data = [d for d in data if d[1] != d[2]]
data.sort(key=lambda row: (row[2]-row[1], row[0].lower()))
print("data len:", len(data))

cyclic_rows = [cyclic_rows[0]] + data

for i in range(1, len(cyclic_rows)):
    if cyclic_rows[i][1] > cyclic_rows[i][2]:
        cyclic_rows[i][0] = f"\\textcolor{{blue}}{{{cyclic_rows[i][0]}}}"
    elif cyclic_rows[i][1] < cyclic_rows[i][2]:
        cyclic_rows[i][0] = f"{cyclic_rows[i][0]}"

table_2 = Texttable()
table_2.set_cols_align(["l"] + ["c" for i in range(len(cyclic_rows[0]) - 1)])
table_2.set_cols_valign(["m" for i in range(len(cyclic_rows[0]))])
table_2.add_rows(cyclic_rows)

print(table_2.draw())

['viktor-relseq', 180, 180, 180, '{\\fontspec{Symbola}\\symbol{"231B}}']
data len: 15
+--------------------+------------------+------------------+
|     Test Name      | WMC cyclic execs | XMM cyclic execs |
+====================+==================+==================+
| assume-ctrl        |        1         |        2         |
+--------------------+------------------+------------------+
| LB+acq             |        0         |        1         |
+--------------------+------------------+------------------+
| LB+coh-cyc         |        0         |        1         |
+--------------------+------------------+------------------+
| LB+ctrl+rel        |        0         |        1         |
+--------------------+------------------+------------------+
| LB+rel             |        0         |        1         |
+--------------------+------------------+------------------+
| LB-invis-write+dep |        0         |        1         |
+--------------------+------------------+------------------+

In [46]:
print(latextable.draw_latex(table_2, position='H', caption=f"Number of cyclic executions found by WMC and GenMC-XMM on litmus tests where XMM has more executions than Weakestmo", label="table:litmus-tests-benchmarks"))

\begin{table}[H]
	\begin{center}
		\begin{tabular}{|l|c|c|}
			\hline
			Test Name & WMC cyclic execs & XMM cyclic execs \\
			\hline
			assume-ctrl & 1 & 2 \\
			\hline
			LB+acq & 0 & 1 \\
			\hline
			LB+coh-cyc & 0 & 1 \\
			\hline
			LB+ctrl+rel & 0 & 1 \\
			\hline
			LB+rel & 0 & 1 \\
			\hline
			LB-invis-write+dep & 0 & 1 \\
			\hline
			po-loc & 0 & 1 \\
			\hline
			java-test9a & 0 & 2 \\
			\hline
			LB+coh-cyc+Wd & 0 & 2 \\
			\hline
			java-test10 & 0 & 3 \\
			\hline
			java-test19 & 3 & 6 \\
			\hline
			java-test20 & 3 & 6 \\
			\hline
			ori & 0 & 3 \\
			\hline
			java-test5 & 4 & 8 \\
			\hline
			LB+coh+RR+cf & 0 & 12 \\
			\hline
		\end{tabular}
	\end{center}
	\caption{Number of cyclic executions found by WMC and GenMC-XMM on litmus tests where XMM has more executions than Weakestmo}
	\label{table:litmus-tests-benchmarks}
\end{table}
